In [1]:
!pip install diffusers --upgrade
!pip install transformers accelerate safetensors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 36.9 MB/s eta 0:00:0000:0100:01


In [2]:
!pip install huggingface_hub
!pip install -U peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 8.6 MB/s eta 0:00:00:00:01


In [3]:
from huggingface_hub.hf_api import HfFolder 
HfFolder.save_token('YOUR HUGGINGFACE TOKEN')

In [6]:
import torch
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import make_image_grid, load_image
from PIL import Image

lora_model_id = "backpropagatorsiitg/LoRA-finetune-muddy"

''' use lora_filename = "pytorch_lora_weights.safetensors" or "pytorch_lora_weights_kohya.safetensors", 
to use method 1 approach where you can use only "muddy underwater style" to generate desired image
use lora_filename = "lora-underwater-10.safetensors", to use method 2 approach where you can 
add variations to your prompts '''

lora_filename = "lora-underwater-10.safetensors"

pipeline = AutoPipelineForImage2Image.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
)
pipeline.enable_model_cpu_offload()
# remove following line if xFormers is not installed or you have PyTorch 2.0 or higher installed
# pipeline.enable_xformers_memory_efficient_attention()
pipeline.load_lora_weights(lora_model_id, weight_name=lora_filename)
pipeline.fuse_lora(lora_scale=1)

prompt = "muddy underwater style"

# Define how many steps and what % of steps to be run on each experts (80/20) here
n_steps = 40

# pass prompt and image to pipeline
image = pipeline(prompt=prompt, num_inference_steps=n_steps, strength=0.8).images[0]
pipeline.unfuse_lora()
make_image_grid([base_image, image], rows=1, cols=2)
image.save("underwater.png")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]